In [13]:
#Imports
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.chains import GraphCypherQAChain
from langchain_groq import ChatGroq
from langchain_community.graphs import Neo4jGraph
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain_community.vectorstores import Neo4jVector
from langchain_core.example_selectors import SemanticSimilarityExampleSelector

import os



In [14]:
NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
GROQ_API_KEY = os.getenv('GROQ_API_KEY')
graph = Neo4jGraph(url =NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD)

In [15]:
graph.refresh_schema()
schema = graph.schema
print(schema)

Node properties:
Year {year: INTEGER}
Month {name: STRING, Rain: FLOAT, month: INTEGER, AF: FLOAT, Tmax: FLOAT, Tmin: FLOAT, Tmean: FLOAT, Sun: FLOAT, Date: DATE}
Relationship properties:

The relationships:
(:Year)-[:HAS_MONTH]->(:Month)


In [16]:
llm = ChatGroq(
    model = "llama-3.1-70b-versatile",
    temperature = 0,
    max_tokens = 2000
)

In [34]:
examples = [
    {
        "question": "Get all weather data for a specific year",
        "query": """MATCH (y:Year {{year:1963}})-[:HAS_MONTH]->(m:Month) 
        RETURN m.name AS Month, m.Tmax AS MaxTemperature, m.Tmin AS MinTemperature,
        m.Rain AS Rainfall, m.Sun AS Sunlight, m.Tmean AS MeanTemperature
        ORDER BY m.month""",
    },
    {
        "question": "what was average temperature in between 1980 and 1990",
        "query": """MATCH (y:Year)-[:HAS_MONTH]->(m:Month)
                    WHERE y.year >= 1980 AND y.year <= 1990
                    RETURN AVG(m.Rain) AS average_rainfall""",
    },
     {
        "question": "what was the most rainy year?",
        "query": """MATCH (y:Year)-[:HAS_MONTH]->(m:Month)
        WITH y.year AS year, AVG(m.Rain) AS avg_rain
        RETURN year, avg_rain
        ORDER BY avg_rain DESC
        LIMIT 1""",
    },
    {
        "question": "In which year we had maximum temperature in may?",
        "query": """MATCH (y:Year)-[:HAS_MONTH]->(m:Month)
        WHERE m.month = 5
        WITH y.year AS year, MAX(m.Tmax) AS max_temp
        RETURN year, max_temp
        ORDER BY max_temp DESC
        LIMIT 1""",
    },

]

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    embeddings,
    Neo4jVector,
    k=2,
    input_keys=["question"],
)



Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (row) { ... }} {position: line: 1, column: 21, offset: 20} for query: "UNWIND $data AS row CALL { WITH row MERGE (c:`Chunk` {id: row.id}) WITH c, row CALL db.create.setNodeVectorProperty(c, 'embedding', row.embedding) SET c.`text` = row.text SET c += row.metadata } IN TRANSACTIONS OF 1000 ROWS "


In [35]:
example_prompt = PromptTemplate.from_template(
    "User input: {question}\nquery: {query}"
)

prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=
    """
    You are a Neo4j expert. Given an input question, create a syntactically correct Cypher query to run.Here is the schema information \nn{schema}.
  
    
    Task:
        You are provided with a natural language query. Your goal is to:
        Understand the user's intent: Identify what the user is asking for, such as retrieving data, finding averages, or comparing values, average between to given dates or months or years
        Generate an appropriate Cypher query: Based on the graph structure and properties provided above, translate the user's natural language query into a Cypher query.
        Return the Cypher query: Ensure the query is correctly formatted and accounts for all relevant properties.

        Use logical operators like >= and <= for filtering node for between specific years, months or dtaes.DO NOT use range() for this
        WHERE clauses should be used for property value comparisons

        
        Given a natural language query, generate a Cypher query that accurately reflects the user's request,
        ensuring that any specific year mentioned in the query is directly translated into the Cypher query. For instance, 
        if the user asks for data from 1991 or in 1991, the Cypher query should explicitly reference the year 1991 in the `MATCH` statement.
        if the user asks for data from january or in january or any other month, the Cypher query should explicitly reference the month anuary or month 1 in the `MATCH` statement.
        ALways give answer obtained from data and never give any suggestion or interpretation  

        Below are a number of examples of questions and their corresponding Cypher queries.",
    """,
    suffix="User input: {question}\nquery: ",
    input_variables=["question", "schema"],
)

In [37]:
print(prompt.format(question="which year had maximum rain in december", schema= schema))


    You are a Neo4j expert. Given an input question, create a syntactically correct Cypher query to run.Here is the schema information 
nNode properties:
Year {year: INTEGER}
Month {name: STRING, Rain: FLOAT, month: INTEGER, AF: FLOAT, Tmax: FLOAT, Tmin: FLOAT, Tmean: FLOAT, Sun: FLOAT, Date: DATE}
Relationship properties:

The relationships:
(:Year)-[:HAS_MONTH]->(:Month).
  
    
    Task:
        You are provided with a natural language query. Your goal is to:
        Understand the user's intent: Identify what the user is asking for, such as retrieving data, finding averages, or comparing values, average between to given dates or months or years
        Generate an appropriate Cypher query: Based on the graph structure and properties provided above, translate the user's natural language query into a Cypher query.
        Return the Cypher query: Ensure the query is correctly formatted and accounts for all relevant properties.

        Use logical operators like >= and <= for filte

In [38]:
chain = GraphCypherQAChain.from_llm(
    graph=graph, llm=llm, cypher_prompt=prompt, verbose=True
)

In [39]:
result = chain.invoke({"query":"which year had maximum rain in december?"})
print(result)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (y:Year)-[:HAS_MONTH]->(m:Month)
        WHERE m.month = 12
        WITH y.year AS year, MAX(m.Rain) AS max_rain
        RETURN year, max_rain
        ORDER BY max_rain DESC
        LIMIT 1
Full Context:
[{'year': 2023, 'max_rain': 170.4}]

> Finished chain.
{'query': 'which year had maximum rain in december?', 'result': 'The year with the maximum rain is 2023.'}


from langchain_core.prompts import ChatPromptTemplate
system_message = """

You are interacting with a Neo4j graph database containing weather data structured as follows:

Graph Structure:
        Year Node (Year)
        Properties:
        year (Integer): 

        Month Node (Month)
        Properties:
        month (Integer): The numeric representation of the month (1 = January, 2 = February, ..., 12 = December).
        name (String): The name of the month (e.g., "January", "February").
        Tmax (Float): Maximum temperature recorded in the month.
        Tmin (Float): Minimum temperature recorded in the month.
        AF (Float): Number of air frosts in the month.
        Rain (Float): Rainfall recorded in millimeters.
        Sun (Float): Sunlight recorded in hours.
        Tmean (Float): Mean temperature in the month.
        Date (Date): Date representation (e.g., "1963-01-01").

        Relationships:
        HAS_MONTH: Connects Year nodes to their respective Month nodes.

Task:
        You are provided with a natural language query. Your goal is to:
        Understand the user's intent: Identify what the user is asking for, such as retrieving data, finding averages, or comparing values, average between to given dates or months or years
        Generate an appropriate Cypher query: Based on the graph structure and properties provided above, translate the user's natural language query into a Cypher query.
        Return the Cypher query: Ensure the query is correctly formatted and accounts for all relevant properties.

        Use logical operators like >= and <= for filtering node for between specific years, months or dtaes.DO NOT use range() for this
        WHERE clauses should be used for property value comparisons 
        


Examples of Natural Language Queries:
        "What was the average temperature in a given year on month?"
        "Show me the month with the most rainfall in given year."
        "Get all the weather data for January across all years."
        "Which year had the highest average temperature in July?"
        "what was the average temperature between two years or months or dates"
        "what was the maximum temperature between two years or months or dates"

Given a natural language query, generate a Cypher query that accurately reflects the user's request,
ensuring that any specific year mentioned in the query is directly translated into the Cypher query. For instance, 
if the user asks for data from 1991 or in 1991, the Cypher query should explicitly reference the year 1991 in the `MATCH` statement.
if the user asks for data from january or in january or any other month, the Cypher query should explicitly reference the month anuary or month 1 in the `MATCH` statement.
ALways give answer obtained from data and never give any suggestion or interpretation

User question: {query}


"""

prompt = ChatPromptTemplate.from_template(system_message)